Пришло время сделать свою БД! Подумайте как организовать базу, создайте её и заполните данными через питон на основе своих данных или воспользуйтесь учебными (https://drive.google.com/open?id=1NWIT8Yn-GdgpBUfFO87dnIDQgmE5nj-j)(30 баллов)


Дополнительное задание!
Напарсите данные для своей БД из интернета) Через запросы, конечно)
(10 доп баллов)